In [1]:
import pyspark as ps
from pyspark.sql import SparkSession

In [43]:
import string
import nltk
from nltk.corpus import stopwords
import pandas as pd

In [2]:
spark1 = SparkSession.builder.appName('Ops').getOrCreate()

In [3]:
path = "yelp_dataset/user.json"
user_df = spark1.read.json(path)
user_df.printSchema()
user_df.createOrReplaceTempView("user")


root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [5]:
path = "yelp_dataset/business.json"
business_df = spark1.read.json(path)
business_df.printSchema()
business_df.createOrReplaceTempView("business")

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [6]:
path = "yelp_dataset/review.json"
review_df = spark1.read.json(path)
review_df.printSchema()
review_df.createOrReplaceTempView("review")

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [17]:
phoenix_1_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and review.stars = 1").toPandas()
# phoenix_2_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and business.stars = 2").toPandas()
# phoenix_3_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and business.stars = 3").toPandas()
# phoenix_4_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and business.stars = 4").toPandas()
# phoenix_5_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and business.stars = 5").toPandas()




In [18]:
phoenix_1_stars.head(10)

,business_id,categories,text
0,XTzKRvWciP_BZ9imk8mmPQ,"Beauty & Spas, Nail Salons",I tried this place because my girls are away f...
1,9MbGG8PxrlpBWVm3QyWy_g,"Health & Medical, Cosmetic Surgeons, Doctors",I am years out from surgery with sacs and I wi...
2,PZMpjUQ9_JVCf4Htu6vZYg,"Hobby Shops, Appliances, Home & Garden, Home D...",((( LADIES BEWARE))) Don't ever take your mach...
3,KjicU7uxRt2KDEnO5cgxDQ,"Shopping, Floral Designers, Event Planning & S...",This company tried deliver flowers to our home...
4,Sxe7h922Dqv9HzfTH-8O7w,"Event Planning & Services, Hotels & Travel, Ho...",One of the crappier Marriotts I've ever stayed...
5,PycR_Mr5jA9jB4Xg3nX0Yw,"Hotels & Travel, Car Rental",They charged me twice what I expected to pay. ...
6,zk3AimWhuz0EZsNgIddKdg,"Food, Coffee & Tea, Juice Bars & Smoothies, Ba...","Awful place. Staff is not nice, very rude. Th..."
7,qaPSbg690KaX5av6xsSV4Q,"Coffee & Tea, Cafes, Food, Tea Rooms, Restaura...","I have no idea what the owner's problem is, bu..."
8,SZQINqa278e7DEM_B3WK8A,"Guns & Ammo, Gold Buyers, Building Supplies, P...",My experience went from bad to worse. This pl...
9,EtLM-iRYp3UGOEz4_b0I2Q,"Hotels & Travel, Car Rental",What a mess. Been in line over an hour and the...


In [19]:
phoenix_1_stars.shape[0]


118901

In [21]:
#lower-casing review text
phoenix_1_stars['text'] = phoenix_1_stars['text'].apply(lambda x: x.lower())
phoenix_1_stars.head()

,business_id,categories,text
0,XTzKRvWciP_BZ9imk8mmPQ,"Beauty & Spas, Nail Salons",i tried this place because my girls are away f...
1,9MbGG8PxrlpBWVm3QyWy_g,"Health & Medical, Cosmetic Surgeons, Doctors",i am years out from surgery with sacs and i wi...
2,PZMpjUQ9_JVCf4Htu6vZYg,"Hobby Shops, Appliances, Home & Garden, Home D...",((( ladies beware))) don't ever take your mach...
3,KjicU7uxRt2KDEnO5cgxDQ,"Shopping, Floral Designers, Event Planning & S...",this company tried deliver flowers to our home...
4,Sxe7h922Dqv9HzfTH-8O7w,"Event Planning & Services, Hotels & Travel, Ho...",one of the crappier marriotts i've ever stayed...


In [25]:
def remove_punc(str):
    table = str.maketrans('', '', string.punctuation)
    words=str.split()
    stripped = [w.translate(table) for w in words]
    return stripped
remove_punc("Hello, this. is a test")

['Hello', 'this', 'is', 'a', 'test']

In [29]:
#Removing punctuation
phoenix_1_stars['text'] = phoenix_1_stars['text'].apply(lambda x: remove_punc(x))


In [30]:
phoenix_1_stars.head(3)

,business_id,categories,text
0,XTzKRvWciP_BZ9imk8mmPQ,"Beauty & Spas, Nail Salons","[i, tried, this, place, because, my, girls, ar..."
1,9MbGG8PxrlpBWVm3QyWy_g,"Health & Medical, Cosmetic Surgeons, Doctors","[i, am, years, out, from, surgery, with, sacs,..."
2,PZMpjUQ9_JVCf4Htu6vZYg,"Hobby Shops, Appliances, Home & Garden, Home D...","[, ladies, beware, dont, ever, take, your, mac..."


In [51]:
#Removing stop words
nltk.download('stopwords')
stop = stopwords.words('english')

def rem_stop_words(txt):
    return ' '.join([word for word in txt if word not in stop])


phoenix_1_stars['text'] = phoenix_1_stars['text'].apply(lambda x: rem_stop_words(x))
phoenix_1_stars['text'].head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maxreynolds/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    tried place girls away month really wanted try...
1    years surgery sacs wish could turn back time p...
2     ladies beware dont ever take machine 35th ave...
3    company tried deliver flowers home son home ca...
4    one crappier marriotts ive ever stayed staff n...
Name: text, dtype: object

In [52]:
phoenix_1_stars['text'][0]

'tried place girls away month really wanted try wait came back nobody hold candle toes desperate went almost twice usually pay begin starting pedicure specifically told already gel polish toes didnt tell right extra 5 waited time pay charged charged kinds extra fees didnt even ask also informed girl toenail loose needed gluing looking phone sudden felt sensation ripped toenail without even asking another girl come put acrylic since nail underneath 12 grown didnt tell much would even ask permission charged end added stone massage barely massage rubbed stones lower legs couple times lasted 30 seconds asked design im used getting board bunch designs choose didnt even ask wanted painted lines two toes charge 5 dollars ugh dont care ugly toes look next time never go anywhere else besides bella nails rock place appalling didnt want make scene paid money never go back'

In [66]:
#Word Frequency
freq = pd.Series(' '.join(phoenix_1_stars['text']).split()).value_counts().sort_values(ascending=False)

In [78]:
for x in freq[:100].iteritems():
    print(x)

('would', 69778)
('get', 63235)
('back', 58273)
('time', 56375)
('service', 55810)
('one', 55526)
('place', 49210)
('said', 46835)
('told', 46670)
('like', 46281)
('never', 45995)
('food', 44412)
('even', 41514)
('dont', 41072)
('go', 40768)
('us', 39190)
('didnt', 34737)
('could', 33042)
('got', 33011)
('asked', 31190)
('car', 30846)
('called', 29499)
('went', 29019)
('good', 28240)
('order', 28037)
('customer', 27865)
('minutes', 26720)
('people', 26526)
('call', 26248)
('im', 25528)
('going', 24925)
('know', 24864)
('came', 24804)
('another', 23994)
('first', 23769)
('day', 22642)
('ever', 22472)
('take', 21877)
('manager', 21826)
('company', 21758)
('still', 21321)
('give', 21000)
('come', 20817)
('took', 20577)
('2', 20480)
('make', 20326)
('really', 19929)
('way', 19898)
('work', 19864)
('two', 19591)
('bad', 19307)
('also', 19304)
('new', 19020)
('experience', 18969)
('money', 18874)
('ordered', 18836)
('well', 18642)
('business', 18537)
('want', 18431)
('left', 18260)
('say', 1